In [22]:
# Import libraries

import time
import pandas as pd
import requests
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
# Set up the API

API = "api-fxpractice.oanda.com"
STREAM_API = "stream-fxpractice.oanda.com"
ACCESS_TOKEN = "06a31b21abc8fb4cbe9bbd8c5b7b1f4e-260aefea008b878b6bffbfebafc84b9f"
ACCOUNT_ID = "101-004-26291813-001"

In [122]:
# Import the nescessary data

# Time format is year, month, day
from_time = time.mktime(pd.to_datetime("2023-07-1-00-00-00").timetuple())
to_time = time.mktime(pd.to_datetime("2023-07-10-00-00-00").timetuple())

header = {"Authorization": "Bearer " + ACCESS_TOKEN}
# Granularity is limited for the amount of data available, no more than 3000 entries give or take
query = {"from": from_time, "to": to_time, "granularity": "M5"}

INSTRUMENT = "XAU_GBP"
CANDLES_PATH = f"/v3/accounts/{ACCOUNT_ID}/instruments/{INSTRUMENT}/candles"

response = requests.get("https://" + API + CANDLES_PATH, headers=header, params=query)

def json_to_pd(json):
    json_file = json.json()
    price_json = json_file["candles"]
    times = []
    close_price, high_price, low_price, open_price = [], [], [], []

    for candle in price_json:
        times.append(candle["time"])
        close_price.append(float(candle["mid"]["c"]))
        high_price.append(float(candle["mid"]["h"]))
        low_price.append(float(candle["mid"]["l"]))
        open_price.append(float(candle["mid"]["o"]))
    
    dataframe = pd.DataFrame({"close": close_price, "high": high_price, "low": low_price, "open": open_price})
    dataframe.index = pd.to_datetime(times)
    return dataframe

get_data = json_to_pd(response)

In [123]:
# Data Preprocessing

# Calculate the percentage change of the data
def calculate_percentage_change(data):
    data['open'] = data['open'].pct_change()
    data['high'] = data['high'].pct_change()
    data['low'] = data['low'].pct_change()
    data['close'] = data['close'].pct_change()

    data.dropna(inplace=True)
    return data
percent_data = calculate_percentage_change(get_data)

# Scale the data
scaler = MinMaxScaler()
dataframe_scaled = pd.DataFrame(scaler.fit_transform(percent_data), columns=percent_data.columns, index=percent_data.index)

# Train, test and validation split
train_end = int(0.8 * len(dataframe_scaled))
val_end = int(0.9 * len(dataframe_scaled))

df_train = dataframe_scaled.iloc[:train_end]
df_val = dataframe_scaled.iloc[train_end: val_end]
df_test = dataframe_scaled.iloc[val_end:]